# Set Repos y Get Proyectos

In [2]:
from jira import JIRA
import json


SPRING = 'https://jira.spring.io'
MONGO = "https://jira.mongodb.org"
APACHE = "https://issues.apache.org/jira"
JBOSS = "https://issues.jboss.org"
ATLASSIAN = 'https://jira.atlassian.com'
#CODEHAUS = ??

options = {
            "server": APACHE,
            "name": "APACHE",
            #"verify": False
          }

jira = JIRA(options)

# Get all projects viewable by anonymous users.
projects = jira.projects()

keys = sorted([project.key for project in projects])[2:5]
names = sorted([project.name for project in projects])

print("Total: ", len(projects))
print("Key -> Name")
print("-----------------")
for project in projects:
    print(project.key, " -> ", project.name, "(" + project.self +")")

    



Total:  658
Key -> Name
-----------------
AGILA  ->   Agila (https://issues.apache.org/jira/rest/api/2/project/10730)
AAR  ->  aardvark (https://issues.apache.org/jira/rest/api/2/project/12320120)
ABDERA  ->  Abdera (https://issues.apache.org/jira/rest/api/2/project/12310505)
ACCUMULO  ->  Accumulo (https://issues.apache.org/jira/rest/api/2/project/12312121)
ACE  ->  ACE (https://issues.apache.org/jira/rest/api/2/project/12310931)
ACL  ->  ActiveCluster (https://issues.apache.org/jira/rest/api/2/project/12311200)
AMQ  ->  ActiveMQ (https://issues.apache.org/jira/rest/api/2/project/12311210)
AMQNET  ->  ActiveMQ .Net (https://issues.apache.org/jira/rest/api/2/project/12311201)
APLO  ->  ActiveMQ Apollo (https://issues.apache.org/jira/rest/api/2/project/12311310)
ARTEMIS  ->  ActiveMQ Artemis (https://issues.apache.org/jira/rest/api/2/project/12315920)
AMQCPP  ->  ActiveMQ C++ Client (https://issues.apache.org/jira/rest/api/2/project/12311207)
AMQCLI  ->  ActiveMQ CLI Tools (https://issu

MAVIBOT  ->  Mavibot (https://issues.apache.org/jira/rest/api/2/project/12314620)
MEECROWAVE  ->  Meecrowave (https://issues.apache.org/jira/rest/api/2/project/12320731)
MESOS  ->  Mesos (https://issues.apache.org/jira/rest/api/2/project/12311242)
METRON  ->  Metron (https://issues.apache.org/jira/rest/api/2/project/12319323)
MILAGRO  ->  Milagro (https://issues.apache.org/jira/rest/api/2/project/12319526)
DIRMINA  ->  MINA (https://issues.apache.org/jira/rest/api/2/project/10670)
SSHD  ->  MINA SSHD (https://issues.apache.org/jira/rest/api/2/project/12310849)
MIRAE  ->  Mirae (https://issues.apache.org/jira/rest/api/2/project/10671)
MNEMONIC  ->  Mnemonic (https://issues.apache.org/jira/rest/api/2/project/12319820)
MODPYTHON  ->  mod_python (https://issues.apache.org/jira/rest/api/2/project/10640)
MRQL  ->  MRQL (https://issues.apache.org/jira/rest/api/2/project/12314428)
MRUNIT  ->  MRUnit (https://issues.apache.org/jira/rest/api/2/project/12311292)
MUSE  ->  Muse (https://issues.apa

# Función para Obetner Issues

In [3]:
def getIssues(project):    
    block_size = 100
    block_num = 0
    allissues = []

    while True:
        start_idx = block_num*block_size
        issues = jira.search_issues("project = '"+project+"'", start_idx, block_size)

        #if len(issues) == 0 or block_num == 5:
        if len(issues) == 0:
            break

        block_num += 1
        
        for issue in issues:
            allissues.append(issue)

    print(len(allissues), ' Issues for Project ', project)
    return allissues

# Función para Obtener Data de Issues

In [4]:

import pandas as pd

def getIssuesData(allissues):
    issues = pd.DataFrame()
    flag = True
    count = 0;
    allComments = pd.DataFrame()

    for issue in allissues: 

        # Check basic params for each issue, if does not has... continue
        if issue.self==None or issue.key==None or issue.fields.created==None or issue.fields.issuetype.name==None:
            continue

        # Get comment for an issue
        comments = jira.comments(issue.key) 

        # Get clean description for an issue
        if(issue.fields.description):
            description = ' '.join(issue.fields.description.split())
        else:
            description = '-'

        # Create object for each issue
        c = {
                'key': issue.key,
                #'description': description,
                'summary': issue.fields.summary,
                'issuetype': issue.fields.issuetype.name,
                'issueResolution': issue.fields.resolution,
                'issueCreated' : issue.fields.created,
                'resolution.date': issue.fields.resolutiondate,
                'self': issue.self
        }

        # Extract comments for each issue
        for comment in comments:
            c['lenComment'] = len(comments)
            c['comment'] = ' '.join(comment.body.split())
            c['author'] = comment.author.name
            c['timeZone'] = comment.author.timeZone
            c['issueComDate'] = comment.created
            allComments = allComments.append(c, ignore_index=True)

    print('Number of comments:', len(allComments))
    return allComments
    #allComments.head(50)


# Función para Guardar Resultados

In [6]:
def saveIssuesData(comments, repo, project):
    print('saved!')
    comments.to_csv('repoData/ISSUES-COMMENTS-' + repo + '-' + project + '.csv', encoding='utf-8', header=True, index=False)
    
    

# Ejecución para Obetener Comentarios de Issues de Proyecto(s)

In [7]:
allComments = pd.DataFrame()

projToSearchFilter = ["HADOOP"]

for project in projects:
    if project.key in projToSearchFilter:
        try:
            allissues = getIssues(project.key)
            allComments = getIssuesData(allissues)  
        except:
            saveIssuesData(allComments, options['name'], project.key)
        
        finally:
            saveIssuesData(allComments, options['name'], project.key)




15037  Issues for Project  HADOOP


{'response': None, 'request': <PreparedRequest [GET]>}\{'response': None, 'request': <PreparedRequest [GET]>}
{'response': None, 'request': <PreparedRequest [GET]>}\{'response': None, 'request': <PreparedRequest [GET]>}


Number of comments: 159907
saved!
